In [82]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import ParameterGrid
from sklearn.pipeline import Pipeline
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix, hstack
import itertools
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing
from sklearn.metrics import coverage_error
from sklearn.svm import SVC
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.neural_network import MLPClassifier
import scipy.sparse

In [2]:
#import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from bs4 import BeautifulSoup

#from sklearn.preprocessing import StandardScaler

from scipy.sparse import coo_matrix, hstack


In [142]:
dataset = pd.read_csv('Cleaned_Posts.csv', nrows = None, index_col=0)
dataset.shape

(50000, 10)

In [143]:
dataset.head()

,TText,TText_NEG,PText,TCode,PCode,TTitle,TTitle_NEG,Title,TagsSet,Tags
Id,,,,,,,,,,
48046688,hi tri instal python packag call pysptk ubuntu...,hi tri instal python packag call pysptk ubuntu...,Hi I was trying to install a python package c...,pip install pysptk / usr / bin / ld : cannot f...,pip install pysptk\n /usr/bin/ld: cannot find ...,tri instal pythonn packag encount `` find -lgc...,tri instal pythonn packag encount `` find_NEG ...,tried to install a pythonn package but encount...,"{'html', 'p', 'code', 'body', 'pre'}",<python><gcc>
48046690,tri make chrome extens display differ text pop...,tri make chrome extens display differ text pop...,I am trying to make a chrome extension that d...,"< script src ="" popup . js ""> </ script >< htm...","<script src=""popup.js""></script>\n<html>\n<hea...",updat chrome extens popup text,updat chrome extens popup text,Update chrome extension popup text,"{'html', 'p', 'code', 'strong', 'body', 'pre'}",<javascript><html><google-chrome-extension>
48046378,call dll python use ctype want pass ctype arra...,call dll python use ctype want pass ctype arra...,"Calling a dll from Python using ctypes, I wan...",ReturnVec = ctypes . c_float * len ( arrA ) t ...,ReturnVec = ctypes.c_float * len(arrA)\nt = ty...,python ctype pass ctype array dll,python ctype pass ctype array dll,Python ctypes: how to pass ctypes array to DLL?,"{'html', 'p', 'code', 'blockquote', 'body', 'p...",<python><ctypes>
48046379,django app user creat video collag use multipl...,django app user creat video collag use multipl...,I have a Django app with which users can crea...,client_max_body_size 100M fastcgi_buffers 16k ...,client_max_body_size 100M\n fastcgi_buffers 8 ...,nginx gunicorn bad gateway upstream prematur c...,nginx gunicorn bad gateway upstream prematur c...,nginx gunicorn 502 bad gateway: upstream prema...,"{'html', 'p', 'code', 'body', 'pre'}",<python><django><nginx>
48046382,'m tri implement hittest use arkit line inters...,'m tri implement hittest use arkit line inters...,I'm trying to implement hittest that is used ...,"std :: pair < bool , double > linePlaneInterse...","std::pair<bool, double> linePlaneIntersection...",implement hit test,implement hit test,Implementing hit test,"{'html', 'p', 'pre', 'code', 'body', 'a'}",<c++><opencv><3d><computer-vision><arkit>


In [5]:
#Main tag loading
def load_obj(name):
    with open('maintaglist.pkl', 'rb') as f:
        return pickle.load(f)
maintaglist = load_obj('maintaglist')
maintaglist

{'.net': 18,
 'ajax': 27,
 'android': 5,
 'angularjs': 20,
 'arrays': 19,
 'asp.net': 14,
 'c': 17,
 'c#': 3,
 'c++': 9,
 'css': 11,
 'html': 8,
 'ios': 10,
 'iphone': 23,
 'java': 2,
 'javascript': 1,
 'jquery': 6,
 'json': 21,
 'mysql': 12,
 'node.js': 25,
 'objective-c': 16,
 'php': 4,
 'python': 7,
 'r': 24,
 'regex': 28,
 'ruby': 26,
 'ruby-on-rails': 15,
 'sql': 13,
 'sql-server': 22,
 'swift': 29,
 'xml': 30}

In [145]:
#Tags parsing on dataset

def maintagclass(entry):
    for tag in entry:
        if tag in maintaglist:
            return tag
    return -1

dataset['PTags'] = dataset['Tags'].apply(lambda x: [tag.name for tag in BeautifulSoup(x, 'html.parser').find_all()])
dataset['MainTag'] = dataset['PTags'].apply(maintagclass)
dataset[['MainTag', 'PTags', 'Tags']].head()

,MainTag,PTags,Tags
Id,,,
48046688,python,"[python, gcc]",<python><gcc>
48046690,javascript,"[javascript, html, google-chrome-extension]",<javascript><html><google-chrome-extension>
48046378,python,"[python, ctypes]",<python><ctypes>
48046379,python,"[python, django, nginx]",<python><django><nginx>
48046382,c++,"[c++, opencv, computer-vision, arkit]",<c++><opencv><3d><computer-vision><arkit>


In [7]:
dataset = dataset.dropna(subset = ['TText', 'PTags'])

In [8]:
#Subset cleaning with only maintags
codedataset = dataset[dataset['MainTag'] != -1]
codedataset.shape

(3250, 12)

In [9]:
codedataset = codedataset.dropna(subset = ['TCode'])
codedataset.shape

(2834, 12)

In [10]:
codedataset[['TCode', 'MainTag']].groupby('MainTag').count()

,TCode
MainTag,
ajax,3
android,219
angularjs,28
arrays,8
asp.net,9
c,52
c#,195
c++,94
css,31


In [36]:
#feature creation to keep the same number of occurences
def datasetbalancing(dataset, index, column, minocc):
    """Returns a list of indexes to balance a dataset"""
    indexcount = dataset[[index, column]].groupby(index).count()[column].to_dict()
    toreturn = []  #Other solution with --> np.full(dataset.shape[0], True, dtype=bool)
    for lang, count in indexcount.items():
        if count >= minocc:
            df = dataset[dataset[index] == lang]
            toreturn.extend(list(df.sample(minocc).index))
            print('we keep {}'.format(lang))
    return toreturn
indexlist = datasetbalancing(codedataset, 'MainTag', 'TCode', 500)

we keep javascript


In [ ]:
codedataset = codedataset.loc[indexlist]

In [11]:
codedataset[['TCode', 'MainTag']].groupby('MainTag').count()

,TCode
MainTag,
ajax,42
android,1905
angularjs,252
arrays,155
asp.net,141
c,450
c#,2158
c++,1029
css,313


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(codedataset['TCode'], codedataset['MainTag'], test_size=0.33, random_state=0, shuffle = True)



In [ ]:
#Pipeline creation

tfidf = TfidfVectorizer(tokenizer=None, vocabulary=None)

nmf = NMF(random_state = 0, init='nndsvd')

rmf = RandomForestClassifier(random_state=0)


pipe = [('tfidf', tfidf), ('nmf', nmf), ('rmf', rmf)]
kpipeline = Pipeline(pipe)

In [ ]:
#Gridsearch creation
parameters = {'tfidf__ngram_range':[(1,1), (1,3)] , 'tfidf__min_df':[.10, .05,], 'tfidf__max_df':[0.8, 1.,],
              'rmf__max_depth':range(10,11,4), 
              'nmf__n_components':range(40,51,50), 'nmf__alpha':[0.05, 0.075], 'nmf__l1_ratio':[0.25, .2],}
scorer = make_scorer(f1_score, average = 'micro')
rmfsearch = GridSearchCV(kpipeline, parameters, n_jobs=-1, cv=2, verbose = 3, scoring = scorer)

In [ ]:
#Fitting
rmfsearch.fit(X_train, y_train)

In [ ]:
rmfsearch.score(X_test, y_test)

In [ ]:
rmfsearch.best_params_

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, rmfsearch.predict(X_test))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, rmfsearch.predict(X_test))

def plot_confusion_matrix(cm,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    #tick_marks = np.arange(len(classes))
    #plt.xticks(tick_marks, classes, rotation=45)
    #plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(confusion_matrix(y_test, rmfsearch.predict(X_test)))

In [ ]:
df = dataset[['TText', 'TTitle']]# dataset['dataset']
df[2:]

In [146]:
#Most commons tags
df = pd.DataFrame(dataset['PTags'].tolist()).stack().value_counts()
#df[:50].plot(kind = 'bar')
#plt.show()

popularitytags = df[df>100].to_dict()
len(popularitytags)

189

In [147]:
textset = dataset.copy()
textset['PTagsCleaned'] = textset['PTags'].apply(lambda x: [tag for tag in x if tag in popularitytags.keys()])
textset['PTagsCleanedLen'] = textset['PTagsCleaned'].apply(lambda x: len(x))
textset = textset[textset['PTagsCleanedLen'] != 0]
textset = textset.dropna()
print(textset.shape)
textset[['PTagsCleaned', 'PTags', 'TCode']].head()

(35536, 14)


,PTagsCleaned,PTags,TCode
Id,,,
48046688,[python],"[python, gcc]",pip install pysptk / usr / bin / ld : cannot f...
48046690,"[javascript, html]","[javascript, html, google-chrome-extension]","< script src ="" popup . js ""> </ script >< htm..."
48046378,[python],"[python, ctypes]",ReturnVec = ctypes . c_float * len ( arrA ) t ...
48046379,"[python, django, nginx]","[python, django, nginx]",client_max_body_size 100M fastcgi_buffers 16k ...
48046382,"[c++, opencv]","[c++, opencv, computer-vision, arkit]","std :: pair < bool , double > linePlaneInterse..."


In [34]:

class CustomLDA(BaseEstimator, TransformerMixin):

    def __init__(self,text_n_components = 10):
        self.text_n_components = text_n_components
    
    def predict(self, X, y=None):
        dftext = X['TText']
        dftitle = X['TTitle']
        dfcode = X['TCode']
        #Text preparation
        textvect = self.textcvect.fit_transform(dftext)
        textlda = self.textlda.fit_transform(textvect)
        #Title preparation
        titlevect = self.titlecvect.fit_transform(dftitle)
        titlelda = self.titlelda.fit_transform(titlevect)
        #Code preparation
        codevect = self.codecvect.fit_transform(dfcode)
        codelda = self.codelda.fit_transform(codevect)
        return hstack((textlda, titlelda, codelda))

    def fit(self, X, y=None):
        dftext = X['TText']
        dftitle = X['TTitle']
        dfcode = X['TCode']
        #Text preparation
        self.textcvect = CountVectorizer(tokenizer=None, vocabulary=None)
        temp = self.textcvect.fit_transform(dftext)
        self.textlda = LatentDirichletAllocation(n_components=5, learning_method = 'batch')
        self.textlda.fit(temp)
        
        #Title preparation
        self.titlecvect = CountVectorizer(tokenizer=None, vocabulary=None, )
        temp = self.titlecvect.fit_transform(dftitle)
        self.titlelda = LatentDirichletAllocation(n_components=5, learning_method = 'batch')
        self.titlelda.fit(temp)
        
        #Code preparation
        self.codecvect = CountVectorizer(tokenizer=None, vocabulary=None, )
        temp = self.codecvect.fit_transform(dfcode)
        self.codelda = LatentDirichletAllocation(n_components=5, learning_method = 'batch')
        self.codelda.fit(temp)
#        return self
    
    def transform(self, X, y=None):
        dftext = X['TText']
        dftitle = X['TTitle']
        dfcode = X['TCode']
        
        #Text preparation
        textvect = self.textcvect.transform(dftext)
        textlda = self.textlda.transform(textvect)
        
        #Title preparation
        titlevect = self.titlecvect.transform(dftitle)
        titlelda = self.titlelda.transform(titlevect)
        return np.hstack((textlda, titlelda))
    
        #Code preparation
        codevect = self.codecvect.transform(dfcode)
        codelda = self.codelda.transform(codevect)
        
        return np.hstack((textlda, titlelda, codelda))
    


customLDA = CustomLDA()
customLDA.fit(textset[['TText', 'TTitle', 'TCode']])
print('Step1 done')
#customLDA.transform(textset[['TText', 'TTitle', 'TCode']])

Step1 done


In [148]:
y = textset['PTagsCleaned']

mlb = preprocessing.MultiLabelBinarizer()
Y = mlb.fit_transform(y)
Y.shape

(35536, 189)

In [149]:
X = textset[['TText', 'TTitle', 'TCode']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=0, shuffle = True)

In [37]:
#Pipeline creation
svm = SVC()

#rmf = RandomForestClassifier(random_state=0)
ovr = OneVsRestClassifier(svm)
scorer = make_scorer(coverage_error)

pipe = [('customlda', CustomLDA()), 
       ('ovr', ovr)
       ] #
kpipeline = Pipeline(pipe)

In [38]:
kpipeline.fit(X_train, y_train)

AttributeError: 'NoneType' object has no attribute 'transform'

In [150]:
X_train

,TText,TTitle,TCode
Id,,,
48047561,applic use work use primefac want upload file ...,upload file use ajax servlet version request.g...,"< h : form id ="" newCustomerformId ""> <! -- re..."
48110629,'m current develop web applic use spring boot ...,use webpack non-front-end librari,spring boot thymeleaf grunt webpack
48080092,saw interest command pars json object use flag...,use kubectl get pod -o=jsonpath=,- o = jsonpath # pod =$ ( kubectl get pods -- ...
48173024,three class folder object item everi folder li...,attempt creat object type item exist primari k...,Folder Object Item Folder objects Object items...
48184590,want save output separ specifi folder follow c...,set directori system.setout,public static void main ( String [] args ) thr...
48073830,tri render partial view _layout.cshtml get err...,html.renderact model pass error,"@ Html . RenderAction ("" _GuestNav "", Categori..."
48120232,user suppos enter one sever given name prompt ...,compar prompt respons variabl,"< html >< body bgcolor ="" firebrick ""> < title..."
48160453,hi guy expand list view bunch item item click ...,android alarm expandablelistview automat updat,@ Override public View getChildView ( int grou...
48129892,tri use scrapi crawl seri game store issu use ...,troubl crawl game store use scrapi html chang ...,allowed_urls =[ ' https :/ / www . cdkeys . co...


In [48]:
#Text preparation
Textcvect = CountVectorizer(tokenizer=None, vocabulary=None, ngram_range = (1,3), min_df=10, max_df=.9)
temp = Textcvect.fit_transform(textset['TText'])
Textlda = LatentDirichletAllocation(n_components=150, learning_method = 'batch')
textlda = Textlda.fit_transform(temp)
textlda.shape

(2766, 150)

In [49]:
#Title preparation
Titlecvect = CountVectorizer(tokenizer=None, vocabulary=None, ngram_range = (1,3), min_df=10, max_df=.9)
temp = Titlecvect.fit_transform(textset['TTitle'])
Titlelda = LatentDirichletAllocation(n_components=150, learning_method = 'batch')
titlelda = Titlelda.fit_transform(temp)
titlelda.shape

(2766, 150)

In [50]:
#Code preparation
Codecvect = CountVectorizer(tokenizer=None, vocabulary=None, ngram_range = (1,3), min_df=10, max_df=.9)
temp = Codecvect.fit_transform(textset['TCode'])
Codelda = LatentDirichletAllocation(n_components=150, learning_method = 'batch')
codelda = Codelda.fit_transform(temp)
titlelda.shape

(2766, 150)

In [42]:
#Data assembly
X = np.hstack((textlda, titlelda, codelda))
X.shape

(2766, 150)

In [43]:
y = textset['PTagsCleaned']

mlb = preprocessing.MultiLabelBinarizer()
Y = mlb.fit_transform(y)
Y.shape

(2766, 18)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=0, shuffle = True)

In [51]:
rmf = RandomForestClassifier(random_state=0, max_depth=15, verbose = 0, n_jobs=3)
ovr = OneVsRestClassifier(rmf,  n_jobs=-2)

In [52]:
#Gridsearch RMF
parameters = {'estimator__min_samples_split':[2,3,5], 'estimator__min_samples_leaf':[1,2,5], 'estimator__n_estimators':[10,20]}
scorer = make_scorer(label_ranking_average_precision_score)
clfrfm = GridSearchCV(ovr, parameters, verbose = 3, cv = 2, scoring = scorer)
clfrfm.fit(X_train, y_train)

Fitting 2 folds for each of 18 candidates, totalling 36 fits
[CV] estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=10 
[CV]  estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=10, score=0.2340584921491046, total=   3.3s
[CV] estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.4s remaining:    0.0s


[CV]  estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=10, score=0.21658267338612683, total=   3.4s
[CV] estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=20 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   10.9s remaining:    0.0s


[CV]  estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=20, score=0.21986995085700384, total=   3.3s
[CV] estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=20 
[CV]  estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=20, score=0.20746340292776377, total=   3.3s
[CV] estimator__min_samples_leaf=1, estimator__min_samples_split=3, estimator__n_estimators=10 
[CV]  estimator__min_samples_leaf=1, estimator__min_samples_split=3, estimator__n_estimators=10, score=0.23043269807023622, total=   3.2s
[CV] estimator__min_samples_leaf=1, estimator__min_samples_split=3, estimator__n_estimators=10 
[CV]  estimator__min_samples_leaf=1, estimator__min_samples_split=3, estimator__n_estimators=10, score=0.22093232541396426, total=   3.3s
[CV] estimator__min_samples_leaf=1, estimator__min_samples_split=3, estimator__n_estimators=20 
[CV]  estimator__min_samples_leaf=1, estimator__min_samples_spli

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  3.3min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=3,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
          n_jobs=-2),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'estimator__min_samples_split': [2, 3, 5], 'estimator__min_samples_leaf': [1, 2, 5], 'estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(label_ranking_average_precision_score),
       verbose=3)

In [55]:
print(clfrfm.best_params_)
print(clfrfm.best_score_)

{'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 5, 'estimator__n_estimators': 10}
0.232071115908


In [56]:
counter = 0
for index, prediction in enumerate(mlb.inverse_transform(clfrfm.predict(X_test))):
    if prediction != ():
        print('Prediction completed :')
        print(prediction)
        print(mlb.inverse_transform(y_test)[index])
        counter += 1
    else:
        print('No Prediction')
        print(mlb.inverse_transform(y_test)[index])
    print()
print('Predicted {}'.format(counter))
print('Empty {}'.format(X_test.shape[0] - counter))

No Prediction
('css', 'html')

Prediction completed :
('javascript', 'reactjs')
('javascript', 'reactjs')

No Prediction
('android', 'ios')

No Prediction
('java',)

No Prediction
('java',)

No Prediction
('javascript', 'reactjs')

No Prediction
('ios',)

Prediction completed :
('javascript',)
('javascript',)

Prediction completed :
('javascript',)
('html', 'javascript', 'jquery')

Prediction completed :
('android',)
('android', 'java')

No Prediction
('php',)

No Prediction
('javascript',)

No Prediction
('php',)

No Prediction
('html',)

No Prediction
('javascript',)

No Prediction
('swift',)

No Prediction
('html', 'java')

No Prediction
('css', 'html')

Prediction completed :
('python',)
('python',)

No Prediction
('android', 'java')

Prediction completed :
('javascript',)
('javascript',)

No Prediction
('c#',)

No Prediction
('python',)

No Prediction
('angular', 'css', 'html', 'javascript', 'jquery')

Prediction completed :
('css', 'javascript')
('c#', 'javascript')

No Predictio


No Prediction
('javascript', 'reactjs')

Prediction completed :
('javascript',)
('mysql', 'php')

No Prediction
('python',)

No Prediction
('php',)

Prediction completed :
('css', 'javascript')
('css', 'jquery')

Prediction completed :
('java',)
('java',)

No Prediction
('javascript', 'reactjs')

Prediction completed :
('python',)
('python',)

No Prediction
('php',)

No Prediction
('ios', 'swift')

No Prediction
('java',)

Prediction completed :
('java',)
('java',)

No Prediction
('javascript', 'node.js')

No Prediction
('python',)

No Prediction
('android',)

Prediction completed :
('reactjs',)
('javascript', 'reactjs')

Prediction completed :
('android',)
('ios', 'swift')

Prediction completed :
('javascript',)
('javascript', 'jquery')

Prediction completed :
('android',)
('android', 'java')

No Prediction
('android',)

No Prediction
('python',)

Prediction completed :
('python',)
('python',)

No Prediction
('css', 'html')

No Prediction
('mysql', 'php')

Prediction completed :
('ph

('python', 'python-3.x')

No Prediction
('angular',)

No Prediction
('css', 'html')

No Prediction
('node.js',)

No Prediction
('php',)

Prediction completed :
('java',)
('java',)

No Prediction
('ios', 'swift')

Prediction completed :
('python',)
('python',)

No Prediction
('python',)

Prediction completed :
('python',)
('python',)

Prediction completed :
('javascript',)
('angular',)

No Prediction
('c++', 'python')

Prediction completed :
('ios',)
('ios', 'swift')

Prediction completed :
('android',)
('android', 'java')

No Prediction
('python',)

No Prediction
('android', 'c++')

Prediction completed :
('php',)
('jquery', 'php')

Prediction completed :
('c#',)
('c#',)

No Prediction
('angular',)

Prediction completed :
('angular',)
('angular', 'javascript')

Prediction completed :
('reactjs',)
('reactjs',)

Prediction completed :
('python',)
('python',)

No Prediction
('c#',)

No Prediction
('javascript',)

Prediction completed :
('c++',)
('c++',)

No Prediction
('android', 'java')


('c#',)

Prediction completed :
('android',)
('android',)

Prediction completed :
('python',)
('python',)

No Prediction
('java',)

Prediction completed :
('html',)
('python',)

Prediction completed :
('android',)
('android',)

No Prediction
('mysql',)

Prediction completed :
('android',)
('android',)

Prediction completed :
('python',)
('python',)

No Prediction
('javascript',)

No Prediction
('python-3.x',)

No Prediction
('python',)

No Prediction
('java',)

Prediction completed :
('python',)
('python',)

No Prediction
('java',)

No Prediction
('php',)

Prediction completed :
('javascript',)
('javascript', 'node.js')

Prediction completed :
('reactjs',)
('reactjs',)

No Prediction
('python',)

Prediction completed :
('python',)
('python',)

No Prediction
('javascript',)

No Prediction
('css', 'html')

No Prediction
('python',)

Prediction completed :
('reactjs',)
('html', 'javascript', 'reactjs')

No Prediction
('javascript', 'node.js')

No Prediction
('javascript', 'reactjs')

No P

In [45]:
from sklearn.svm import LinearSVC
svm = SVC()
ovrsvm = OneVsRestClassifier(svm,  n_jobs=-2)

In [54]:
#Gridsearch SVC
parameters = {'estimator__C':np.logspace(-3, 2, 6), 'estimator__gamma':np.logspace(-3, 2, 3)}
scorer = make_scorer(label_ranking_average_precision_score)
svc = SVC()
clf = GridSearchCV(ovrsvm, parameters, verbose = 3, cv = 2, scoring = scorer)
clf.fit(X_train, y_train)

Fitting 2 folds for each of 18 candidates, totalling 36 fits
[CV] estimator__C=0.001, estimator__gamma=0.001 ......................
[CV]  estimator__C=0.001, estimator__gamma=0.001, score=0.07185664629030443, total=   1.3s
[CV] estimator__C=0.001, estimator__gamma=0.001 ......................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s


[CV]  estimator__C=0.001, estimator__gamma=0.001, score=0.07265418766498793, total=   1.2s
[CV] estimator__C=0.001, estimator__gamma=0.316227766017 .............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.7s remaining:    0.0s


[CV]  estimator__C=0.001, estimator__gamma=0.316227766017, score=0.07185664629030443, total=   1.2s
[CV] estimator__C=0.001, estimator__gamma=0.316227766017 .............
[CV]  estimator__C=0.001, estimator__gamma=0.316227766017, score=0.07265418766498793, total=   1.1s
[CV] estimator__C=0.001, estimator__gamma=100.0 ......................
[CV]  estimator__C=0.001, estimator__gamma=100.0, score=0.07185664629030443, total=   1.5s
[CV] estimator__C=0.001, estimator__gamma=100.0 ......................
[CV]  estimator__C=0.001, estimator__gamma=100.0, score=0.07265418766498793, total=   1.4s
[CV] estimator__C=0.01, estimator__gamma=0.001 .......................
[CV]  estimator__C=0.01, estimator__gamma=0.001, score=0.07185664629030443, total=   1.1s
[CV] estimator__C=0.01, estimator__gamma=0.001 .......................
[CV]  estimator__C=0.01, estimator__gamma=0.001, score=0.07265418766498793, total=   1.2s
[CV] estimator__C=0.01, estimator__gamma=0.316227766017 ..............
[CV]  estima

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  2.2min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=-2),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'estimator__C': array([  1.00000e-03,   1.00000e-02,   1.00000e-01,   1.00000e+00,
         1.00000e+01,   1.00000e+02]), 'estimator__gamma': array([  1.00000e-03,   3.16228e-01,   1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(label_ranking_average_precision_score),
       verbose=3)

In [47]:
print(clf.best_params_)
print(clf.best_score_)

{'estimator__C': 100.0, 'estimator__gamma': 0.10000000000000001}
0.434694489417


In [128]:
counter = 0
for index, prediction in enumerate(mlb.inverse_transform(clf.predict(X_test))):
    if prediction != ():
        print('index {} - Prediction completed :'.format(index))
        print(prediction)
        print(mlb.inverse_transform(y_test)[index])
        counter += 1
    else:
        print('index {} - No Prediction'.format(index))
        print(mlb.inverse_transform(y_test)[index])
    print()
print('Predicted {}'.format(counter))
print('Empty {}'.format(X_test.shape[0] - counter))

index 0 - Prediction completed :
('html', 'javascript', 'jquery')
('css', 'html')

index 1 - Prediction completed :
('javascript', 'reactjs')
('javascript', 'reactjs')

index 2 - Prediction completed :
('jquery', 'swift')
('android', 'ios')

index 3 - Prediction completed :
('java',)
('java',)

index 4 - No Prediction
('java',)

index 5 - No Prediction
('javascript', 'reactjs')

index 6 - Prediction completed :
('ios', 'swift')
('ios',)

index 7 - Prediction completed :
('javascript',)
('javascript',)

index 8 - Prediction completed :
('javascript',)
('html', 'javascript', 'jquery')

index 9 - Prediction completed :
('android',)
('android', 'java')

index 10 - Prediction completed :
('php',)
('php',)

index 11 - Prediction completed :
('javascript',)
('javascript',)

index 12 - Prediction completed :
('php',)
('php',)

index 13 - No Prediction
('html',)

index 14 - No Prediction
('javascript',)

index 15 - Prediction completed :
('ios', 'swift')
('swift',)

index 16 - Prediction comple

('css', 'node.js')

index 176 - Prediction completed :
('html', 'javascript')
('html', 'javascript')

index 177 - Prediction completed :
('mysql',)
('mysql', 'php')

index 178 - Prediction completed :
('android',)
('android',)

index 179 - Prediction completed :
('python',)
('python',)

index 180 - No Prediction
('python-3.x',)

index 181 - Prediction completed :
('css', 'html', 'javascript')
('javascript',)

index 182 - Prediction completed :
('python', 'python-3.x')
('python',)

index 183 - Prediction completed :
('c#',)
('python',)

index 184 - Prediction completed :
('css',)
('javascript', 'jquery')

index 185 - Prediction completed :
('android',)
('android', 'java')

index 186 - Prediction completed :
('php',)
('php',)

index 187 - Prediction completed :
('c++',)
('c++',)

index 188 - Prediction completed :
('java',)
('java',)

index 189 - No Prediction
('html',)

index 190 - Prediction completed :
('c#',)
('c#', 'jquery')

index 191 - Prediction completed :
('html',)
('css',)

in

('android',)

index 349 - No Prediction
('javascript', 'jquery')

index 350 - No Prediction
('reactjs',)

index 351 - No Prediction
('r',)

index 352 - Prediction completed :
('python',)
('python',)

index 353 - Prediction completed :
('c#',)
('c#',)

index 354 - No Prediction
('css',)

index 355 - Prediction completed :
('android',)
('android',)

index 356 - No Prediction
('javascript',)

index 357 - Prediction completed :
('c#',)
('c#',)

index 358 - Prediction completed :
('c#',)
('java',)

index 359 - Prediction completed :
('php',)
('php',)

index 360 - Prediction completed :
('node.js',)
('html', 'javascript', 'node.js')

index 361 - No Prediction
('java',)

index 362 - No Prediction
('python',)

index 363 - Prediction completed :
('mysql',)
('mysql',)

index 364 - Prediction completed :
('php',)
('php',)

index 365 - Prediction completed :
('android', 'java')
('android',)

index 366 - Prediction completed :
('c++',)
('c++',)

index 367 - Prediction completed :
('python',)
('pyth


index 517 - Prediction completed :
('angular', 'javascript')
('angular',)

index 518 - Prediction completed :
('c++',)
('c++', 'python')

index 519 - Prediction completed :
('ios',)
('ios', 'swift')

index 520 - Prediction completed :
('android', 'html')
('android', 'java')

index 521 - Prediction completed :
('python',)
('python',)

index 522 - No Prediction
('android', 'c++')

index 523 - Prediction completed :
('mysql', 'php')
('jquery', 'php')

index 524 - Prediction completed :
('c#',)
('c#',)

index 525 - Prediction completed :
('java',)
('angular',)

index 526 - Prediction completed :
('angular',)
('angular', 'javascript')

index 527 - Prediction completed :
('reactjs',)
('reactjs',)

index 528 - Prediction completed :
('python',)
('python',)

index 529 - No Prediction
('c#',)

index 530 - Prediction completed :
('javascript', 'reactjs')
('javascript',)

index 531 - Prediction completed :
('c++',)
('c++',)

index 532 - No Prediction
('android', 'java')

index 533 - Prediction c

('python',)

index 673 - Prediction completed :
('python',)
('python-3.x',)

index 674 - Prediction completed :
('css', 'html')
('css', 'html')

index 675 - Prediction completed :
('html', 'javascript')
('javascript',)

index 676 - Prediction completed :
('c++',)
('c++',)

index 677 - No Prediction
('c#',)

index 678 - Prediction completed :
('ios', 'swift')
('ios', 'swift')

index 679 - No Prediction
('c#',)

index 680 - No Prediction
('jquery',)

index 681 - Prediction completed :
('android', 'java')
('android', 'java')

index 682 - Prediction completed :
('css', 'html', 'javascript', 'jquery')
('css', 'html')

index 683 - Prediction completed :
('android',)
('android', 'php')

index 684 - Prediction completed :
('android', 'java')
('android',)

index 685 - Prediction completed :
('php',)
('python',)

index 686 - No Prediction
('python',)

index 687 - Prediction completed :
('javascript', 'php')
('php',)

index 688 - Prediction completed :
('mysql',)
('javascript',)

index 689 - No P

('php',)

index 824 - Prediction completed :
('python',)
('python',)

index 825 - Prediction completed :
('javascript', 'jquery')
('jquery',)

index 826 - Prediction completed :
('javascript', 'node.js')
('javascript', 'node.js')

index 827 - No Prediction
('android',)

index 828 - Prediction completed :
('node.js',)
('node.js',)

index 829 - Prediction completed :
('javascript',)
('javascript',)

index 830 - Prediction completed :
('php',)
('php',)

index 831 - Prediction completed :
('angular',)
('angular',)

index 832 - Prediction completed :
('mysql', 'php')
('java',)

index 833 - Prediction completed :
('css',)
('css', 'html')

index 834 - Prediction completed :
('r',)
('mysql', 'r')

index 835 - Prediction completed :
('html', 'javascript')
('android',)

index 836 - No Prediction
('angular',)

index 837 - Prediction completed :
('angular',)
('android',)

index 838 - Prediction completed :
('node.js',)
('jquery', 'node.js')

index 839 - Prediction completed :
('python',)
('python'

In [144]:
#Gridsearch perceptron
perceptron = MLPClassifier(random_state=0, max_iter=300)
ovrpercep = OneVsRestClassifier(perceptron,  n_jobs=-2)
parameters = {'estimator__solver':['sgd', 'adam'], 'estimator__alpha':np.logspace(-3, 2, 6), 'estimator__hidden_layer_sizes':[(50, ),(100, ),(200, ),]}
scorer = make_scorer(label_ranking_average_precision_score)
clfpercep = GridSearchCV(ovrpercep, parameters, verbose = 3, cv = 2, scoring = scorer)
clfpercep.fit(X_train, y_train)

Fitting 2 folds for each of 36 candidates, totalling 72 fits
[CV] estimator__alpha=0.001, estimator__hidden_layer_sizes=(50,), estimator__solver=sgd 


/Users/cricket/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/cricket/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  estimator__alpha=0.001, estimator__hidden_layer_sizes=(50,), estimator__solver=sgd, score=0.07185664629030443, total=  10.2s
[CV] estimator__alpha=0.001, estimator__hidden_layer_sizes=(50,), estimator__solver=sgd 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.4s remaining:    0.0s
/Users/cricket/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  estimator__alpha=0.001, estimator__hidden_layer_sizes=(50,), estimator__solver=sgd, score=0.07265418766498793, total=   9.6s
[CV] estimator__alpha=0.001, estimator__hidden_layer_sizes=(50,), estimator__solver=adam 


...

[CV]  estimator__alpha=100.0, estimator__hidden_layer_sizes=(200,), estimator__solver=adam, score=0.07185664629030443, total=  27.8s
[CV] estimator__alpha=100.0, estimator__hidden_layer_sizes=(200,), estimator__solver=adam 


/Users/cricket/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/cricket/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/cricket/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/cricket/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't co

[CV]  estimator__alpha=100.0, estimator__hidden_layer_sizes=(200,), estimator__solver=adam, score=0.07265418766498793, total=  28.5s


[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 20.7min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=OneVsRestClassifier(estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=300, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False),
          n_jobs=-2),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'estimator__solver': ['sgd', 'adam'], 'estimator__alpha': array([  1.00000e-03,   1.00000e-02,   1.00000e-01,   1.00000e+00,
         1.00000e+01,   1.00000e+02]), 'estimator__hidden_layer_sizes': [(50,), (100,), (200,)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(label_ranking_average_precision_score),
       verbose=3)

In [145]:
print(clfpercep.best_params_)
print(clfpercep.best_score_)

{'estimator__alpha': 0.10000000000000001, 'estimator__hidden_layer_sizes': (200,), 'estimator__solver': 'adam'}
0.478008634647


In [146]:
counter = 0
for index, prediction in enumerate(mlb.inverse_transform(clfpercep.predict(X_test))):
    if prediction != ():
        print('index {} - Prediction completed :'.format(index))
        print(prediction)
        print(mlb.inverse_transform(y_test)[index])
        counter += 1
    else:
        print('index {} - No Prediction'.format(index))
        print(mlb.inverse_transform(y_test)[index])
    print()
print('Predicted {}'.format(counter))
print('Empty {}'.format(X_test.shape[0] - counter))

index 0 - Prediction completed :
('css', 'html', 'javascript')
('css', 'html')

index 1 - Prediction completed :
('javascript', 'reactjs')
('javascript', 'reactjs')

index 2 - No Prediction
('android', 'ios')

index 3 - Prediction completed :
('java',)
('java',)

index 4 - No Prediction
('java',)

index 5 - No Prediction
('javascript', 'reactjs')

index 6 - Prediction completed :
('ios', 'swift')
('ios',)

index 7 - Prediction completed :
('javascript',)
('javascript',)

index 8 - Prediction completed :
('javascript',)
('html', 'javascript', 'jquery')

index 9 - Prediction completed :
('android',)
('android', 'java')

index 10 - Prediction completed :
('php',)
('php',)

index 11 - Prediction completed :
('javascript',)
('javascript',)

index 12 - Prediction completed :
('php',)
('php',)

index 13 - No Prediction
('html',)

index 14 - Prediction completed :
('css',)
('javascript',)

index 15 - Prediction completed :
('ios', 'swift')
('swift',)

index 16 - Prediction completed :
('java',

('android',)

index 170 - Prediction completed :
('java',)
('java',)

index 171 - Prediction completed :
('python',)
('python',)

index 172 - Prediction completed :
('javascript', 'jquery')
('jquery',)

index 173 - No Prediction
('javascript',)

index 174 - Prediction completed :
('php',)
('javascript',)

index 175 - No Prediction
('css', 'node.js')

index 176 - Prediction completed :
('html', 'javascript')
('html', 'javascript')

index 177 - Prediction completed :
('mysql',)
('mysql', 'php')

index 178 - Prediction completed :
('android',)
('android',)

index 179 - Prediction completed :
('python',)
('python',)

index 180 - No Prediction
('python-3.x',)

index 181 - Prediction completed :
('html', 'javascript')
('javascript',)

index 182 - Prediction completed :
('python', 'python-3.x')
('python',)

index 183 - Prediction completed :
('c#',)
('python',)

index 184 - Prediction completed :
('css', 'html')
('javascript', 'jquery')

index 185 - Prediction completed :
('android',)
('andro

('c++',)

index 343 - Prediction completed :
('c#',)
('c#',)

index 344 - Prediction completed :
('css',)
('css', 'javascript')

index 345 - Prediction completed :
('java',)
('java',)

index 346 - Prediction completed :
('c#',)
('c#',)

index 347 - Prediction completed :
('java',)
('java',)

index 348 - Prediction completed :
('android',)
('android',)

index 349 - No Prediction
('javascript', 'jquery')

index 350 - No Prediction
('reactjs',)

index 351 - No Prediction
('r',)

index 352 - Prediction completed :
('python',)
('python',)

index 353 - Prediction completed :
('c#',)
('c#',)

index 354 - No Prediction
('css',)

index 355 - Prediction completed :
('android',)
('android',)

index 356 - No Prediction
('javascript',)

index 357 - Prediction completed :
('c#',)
('c#',)

index 358 - Prediction completed :
('c#',)
('java',)

index 359 - Prediction completed :
('php',)
('php',)

index 360 - Prediction completed :
('node.js',)
('html', 'javascript', 'node.js')

index 361 - No Predicti

('java',)

index 496 - Prediction completed :
('r',)
('r',)

index 497 - Prediction completed :
('javascript', 'reactjs')
('javascript', 'reactjs')

index 498 - No Prediction
('java',)

index 499 - Prediction completed :
('html',)
('css', 'html')

index 500 - No Prediction
('android',)

index 501 - Prediction completed :
('python',)
('java',)

index 502 - No Prediction
('angular', 'javascript')

index 503 - Prediction completed :
('python',)
('python',)

index 504 - Prediction completed :
('python',)
('java',)

index 505 - No Prediction
('php',)

index 506 - Prediction completed :
('android',)
('android',)

index 507 - Prediction completed :
('python',)
('python', 'python-3.x')

index 508 - Prediction completed :
('javascript', 'jquery')
('angular',)

index 509 - Prediction completed :
('css',)
('css', 'html')

index 510 - Prediction completed :
('javascript', 'node.js')
('node.js',)

index 511 - Prediction completed :
('php',)
('php',)

index 512 - Prediction completed :
('java',)
('j

('php',)

index 649 - No Prediction
('php',)

index 650 - Prediction completed :
('android',)
('android', 'java')

index 651 - Prediction completed :
('python',)
('python',)

index 652 - Prediction completed :
('javascript',)
('jquery',)

index 653 - No Prediction
('java',)

index 654 - Prediction completed :
('c#',)
('java',)

index 655 - Prediction completed :
('python',)
('ios', 'swift')

index 656 - Prediction completed :
('javascript',)
('html', 'javascript')

index 657 - No Prediction
('javascript',)

index 658 - Prediction completed :
('javascript', 'php')
('php',)

index 659 - Prediction completed :
('css', 'html')
('html', 'php')

index 660 - Prediction completed :
('python', 'python-3.x')
('python',)

index 661 - Prediction completed :
('java',)
('java',)

index 662 - Prediction completed :
('android',)
('ios',)

index 663 - No Prediction
('c#',)

index 664 - Prediction completed :
('java',)
('java',)

index 665 - Prediction completed :
('python',)
('python',)

index 666 - Pr

('javascript',)

index 806 - Prediction completed :
('php',)
('php',)

index 807 - Prediction completed :
('node.js',)
('javascript', 'node.js')

index 808 - Prediction completed :
('reactjs',)
('javascript', 'reactjs')

index 809 - Prediction completed :
('android',)
('android',)

index 810 - Prediction completed :
('python',)
('python',)

index 811 - Prediction completed :
('angular',)
('angular',)

index 812 - Prediction completed :
('python', 'python-3.x')
('python-3.x',)

index 813 - Prediction completed :
('python',)
('python',)

index 814 - No Prediction
('python',)

index 815 - Prediction completed :
('c#',)
('c#',)

index 816 - Prediction completed :
('php',)
('angular',)

index 817 - Prediction completed :
('python', 'r')
('r',)

index 818 - Prediction completed :
('python',)
('ios', 'swift')

index 819 - No Prediction
('python',)

index 820 - No Prediction
('angular',)

index 821 - Prediction completed :
('c++', 'java')
('c++',)

index 822 - Prediction completed :
('c++',)
(

http://scikit-learn.org/stable/modules/multiclass.html#multilabel-classification-format

In [58]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.linear_model import RidgeClassifierCV

In [151]:
class Customtfidf(BaseEstimator, TransformerMixin):

    def __init__(self, text_max_features=None, title_max_features=None, code_max_features=None,
                       text_min_df=1, title_min_df=1, code_min_df=1,
                       text_max_df=1., title_max_df=1., code_max_df=1.,
                       text_ngram_range=(1,1), title_ngram_range=(1,1), code_ngram_range=(1,1), ):
        
        self.text_max_features = text_max_features
        self.title_max_features = title_max_features
        self.code_max_features = code_max_features
        
        self.text_min_df = text_min_df
        self.title_min_df = title_min_df
        self.code_min_df = code_min_df
        
        self.text_max_df = text_max_df
        self.title_max_df = title_max_df
        self.code_max_df = code_max_df
        
        self.text_ngram_range = text_ngram_range
        self.title_ngram_range = title_ngram_range
        self.code_ngram_range = code_ngram_range
    

    def fit(self, X, y=None, **params):
        dftext = X['TText']
        dftitle = X['TTitle']
        dfcode = X['TCode']
        
        #Text preparation
        self.texttfidf = TfidfVectorizer(tokenizer=None, vocabulary=None, max_features=self.text_max_features, min_df=self.text_min_df, max_df=self.text_max_df, ngram_range=self.text_ngram_range)
        self.texttfidf.fit(dftext)
        
        #Title preparation
        self.titletfidf = CountVectorizer(tokenizer=None, vocabulary=None, max_features=self.title_max_features, min_df=self.title_min_df, max_df=self.title_max_df, ngram_range=self.title_ngram_range)
        self.titletfidf.fit(dftitle)

        
        #Code preparation
        self.codetfidf = CountVectorizer(tokenizer=None, vocabulary=None, max_features=self.code_max_features, min_df=self.code_min_df , max_df=self.code_max_df, ngram_range=self.code_ngram_range)
        self.codetfidf.fit(dfcode)
        return self
    
    def transform(self, X, y=None):
        dftext = X['TText']
        dftitle = X['TTitle']
        dfcode = X['TCode']
        
        #Text preparation
        texttfidf = self.texttfidf.transform(dftext)
        
        #Title preparation
        titletfidf = self.titletfidf.transform(dftitle)
    
        #Code preparation
        codetfidf = self.codetfidf.transform(dfcode)
        
        return scipy.sparse.hstack((texttfidf, titletfidf, codetfidf), format='csr')
    


customtfidf = Customtfidf()
#Customtfidf.fit(textset[['TText', 'TTitle', 'TCode']])
#print('Step1 done')
#customtfidf.fit_transform(textset[['TText', 'TTitle', 'TCode']]).shape

In [152]:
#Pipeline creation
svm = SVC()

#rmf = RandomForestClassifier(random_state=0)
ovr = OneVsRestClassifier(svm)
scorer = make_scorer(coverage_error)

pipe = [('customtfidf', Customtfidf()), 
       ('ovr', ovr)
       ]
kpipeline = Pipeline(pipe)
#kpipeline.fit(X_train, y_train)

In [153]:
kpipeline.steps

[('customtfidf',
  Customtfidf(code_max_df=1.0, code_max_features=None, code_min_df=1,
        code_ngram_range=(1, 1), text_max_df=1.0, text_max_features=None,
        text_min_df=1, text_ngram_range=(1, 1), title_max_df=1.0,
        title_max_features=None, title_min_df=1, title_ngram_range=(1, 1))),
 ('ovr',
  OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False),
            n_jobs=1))]

In [155]:
#Gridsearch SVC
#parameters = {'ovr__estimator__C':np.logspace(-3, 2, 6),
#              'ovr__estimator__gamma':np.logspace(-3, 2, 3),
#             }
parameters = {'customtfidf__code_max_features':[10000, 20000,None],
              'customtfidf__text_max_features':[20000,30000, None],
              'customtfidf__title_max_features':[15000,None],
              #'customtfidf__text_min_df':[1,5,10],
              #'customtfidf__title_min_df':[1,5,10],
              #'customtfidf__code_min_df':[1,5,10],
              #'customtfidf__text_max_df':[1.,8.],
              #'customtfidf__title_max_df':[1.,8.],
              #'customtfidf__code_max_df':[1.,8.],
              #'customtfidf__text_ngram_range':[(1,1), (1,2), (1,3)],
              #'customtfidf__title_ngram_range':[(1,1), (1,2), (1,3)],
              #'customtfidf__code_ngram_range':[(1,1), (1,2), (1,3)],
              #'ovr__estimator__C':np.logspace(-3, 2, 2),
              #'ovr__estimator__gamma':np.logspace(-3, 2, 2),
              }

scorer = make_scorer(label_ranking_average_precision_score)
svc = SVC()
clf = GridSearchCV(kpipeline, parameters, verbose = 3, cv = 2, scoring = scorer)
clf.fit(X_train, y_train)

Fitting 2 folds for each of 18 candidates, totalling 36 fits
[CV] customtfidf__code_max_features=10000, customtfidf__text_max_features=20000, customtfidf__title_max_features=15000 
[CV]  customtfidf__code_max_features=10000, customtfidf__text_max_features=20000, customtfidf__title_max_features=15000, score=0.02938249012797348, total=22.3min
[CV] customtfidf__code_max_features=10000, customtfidf__text_max_features=20000, customtfidf__title_max_features=15000 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 33.6min remaining:    0.0s


KeyboardInterrupt: 

In [ ]:
clf.score(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
clf.best_params_